# Imports and Functions

In [ ]:
import requests
requests.packages.urllib3.disable_warnings()
import json
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt
from ternarpy import *
import pandas as pd
import pprint
import numpy
from skbeam.core.fitting.background import snip_method
from scipy import signal

In [ ]:
def check_response(r,quiet=False):
    try:
        r_content = r.json(object_pairs_hook=OrderedDict)
    except:
        r_content = r.text
    if str(r.status_code)[0] is not "2":
        if not quiet: print("Error: ",r.status_code) 
        if not quiet: print(r.text)
    else:
        return r_content

In [ ]:
def mdcs_query(host,query,format='json'):
    url = host.strip("/") + "/rest/explore/query-by-example"
    data = dict()
    data['query'] = query
    data['dataformat'] = format
    r = requests.post(url, data=data, verify=False) # This needs to be fixed
    return check_response(r)

# Get the data

This may break in the future if future round robin datasets are added, which would require more specific queries.

In [ ]:
host = "https://hte.registry.nist.gov/"
query = '{"content.library.keyword":"round robin"}'
query_response = mdcs_query(host,query)

In [ ]:
libraries = dict()
id_list = dict()
id_list['s_nist_c_nist'] = list()
id_list['s_nist_c_nrel'] = list()
id_list['s_nrel_c_nist'] = list()
id_list['s_nrel_c_nrel'] = list()
flip = {'a':'b','b':'a'}
library_pairs = dict()
for record in query_response:
    lib_id = record['content']['library']['identifier'][0]['id']
    org = record['content']['library']['associated']['organization']['altName']
    comment = record['content']['library']['comment']
    libraries[lib_id] = record['content']['library']
    if 'NIST' in comment:
        nist_id = lib_id
        end_char = nist_id[-1]
        nrel_id = nist_id[0:-1]+flip[end_char]
        library_pairs[nist_id] = nrel_id
    
    if 'NIST' in org and 'NIST' in comment:
        id_list['s_nist_c_nist'].append(lib_id)
    if 'NIST' in org and 'NREL' in comment:
        id_list['s_nist_c_nrel'].append(lib_id)
    if 'NREL' in org and 'NIST' in comment:
        id_list['s_nrel_c_nist'].append(lib_id)
    if 'NREL' in org and 'NREL' in comment:
        id_list['s_nrel_c_nrel'].append(lib_id)

print(id_list['s_nist_c_nist'])
print(id_list['s_nist_c_nrel'])
print(id_list['s_nrel_c_nist'])
print(id_list['s_nrel_c_nrel'])

print(library_pairs)

In [ ]:
diffraction_data_records = dict()
for lib_id,library in libraries.items():
    # Note it is possible to get an access URL from the library record. 
    # However, by performing a query for diffraction data, we can reduce
    # the number of requests from one per diffraction pattern to one
    # per library.
    query = '{"content.library-xray-diffraction-measurement.sample.identifier.id":"'+lib_id+'"}'
    query_response = mdcs_query(host,query)
    print(lib_id,len(query_response))
    diffraction_data_records[lib_id] = list()
    for record in query_response:
        diffraction_data_records[lib_id].append(record['content']['library-xray-diffraction-measurement'])

In [ ]:
theta_min = 30.0
theta_max = 40.0
diffraction_spectra_bg_sub_x = dict()
diffraction_spectra_bg_sub_y = dict()
for lib_id,diffraction_records in diffraction_data_records.items():
    diffraction_spectra_bg_sub_x[lib_id] = [None] * len(diffraction_records)
    diffraction_spectra_bg_sub_y[lib_id] = [None] * len(diffraction_records)
    for record in diffraction_records:
        map_index = record['point']['identifier']['id']
        n = map_index - 1
        pattern = record['data']
        x = numpy.array(pattern['angle-2-theta']['value'])
        y = numpy.array(pattern['intensity']['value'])
        condition = np.logical_and(x >= theta_min, x <= theta_max)
        x_range = x[condition]
        bg = snip_method(y,0.0,1.0,0.0,iter_num=20)
        y_bg_sub = y - bg
        y_range_bg_sub = y_bg_sub[condition]
        y_norm = y_range_bg_sub/max(y_range_bg_sub)
        diffraction_spectra_bg_sub_x[lib_id][n] = list(x_range)
        diffraction_spectra_bg_sub_y[lib_id][n] = list(y_norm)

In [ ]:
for lib_id in diffraction_spectra_bg_sub_x:
    plt.figure(figsize=(10,10))
    img = diffraction_spectra_bg_sub_y[lib_id]
    map_id_l = 1
    map_id_u = len(img)
    theta_l = diffraction_spectra_bg_sub_x[lib_id][0][0]
    theta_u = diffraction_spectra_bg_sub_x[lib_id][0][-1]

    plt.imshow(img, cmap='coolwarm', interpolation='nearest', 
               aspect=(theta_u-theta_l)/(map_id_u-map_id_l),
               extent=[theta_l,theta_u,map_id_l,map_id_u])
    plt.xlabel('2θ (deg)')
    plt.ylabel('Map Position')
    plt.title(lib_id)
    f_name = 'xrd-range-'+lib_id.replace('20.500.12048/','')+'.pdf'
    plt.savefig(f_name, bbox_inches='tight')
    plt.show()


In [ ]:
theta_width = 5.0
primary_peak_data = dict()
for lib_id in diffraction_spectra_bg_sub_x:
    primary_peak_data[lib_id] = dict()
    primary_peak_data[lib_id]['theta'] = list()
    primary_peak_data[lib_id]['intensity'] = list()
    for i,y in enumerate(diffraction_spectra_bg_sub_y[lib_id]):
        x = numpy.array(diffraction_spectra_bg_sub_x[lib_id][i])
        y = numpy.array(y)
        dx = x[1]-x[0]
        N_width = int(theta_width/dx)
        peaks = signal.find_peaks_cwt(y,numpy.arange(1,N_width))
        y_max = 0.0
        primary_peak = None
        for peak in peaks:
            if y[peak] > y_max:
                primary_peak = peak
        primary_peak_theta = x[primary_peak]
        primary_peak_intensity = y[primary_peak]
        primary_peak_data[lib_id]['theta'].append(primary_peak_theta)
        primary_peak_data[lib_id]['intensity'].append(primary_peak_intensity)

In [ ]:
colors = 'krgb'
font = {'size'   : 18}
matplotlib.rc('font', **font)
plt.figure(figsize=(10,10))
i = 0
for nist_id,nrel_id in library_pairs.items():
    plt.scatter(
        primary_peak_data[nrel_id]['theta'],
        primary_peak_data[nist_id]['theta'],
        c=colors[i], 
        label = nist_id.split('/')[1][0:-1])
    i = i+1

plt.plot([theta_min,theta_max],[theta_min,theta_max],'k--')
plt.legend()
plt.axes().set_aspect('equal')
ax = plt.axes()

plt.grid(True)
plt.show()